Import dependencies

In [1]:
import numpy as np
import os
import pandas as pd

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

Directory

In [2]:
os.chdir(os.getcwd()+'/images')
print(os.getcwd())

C:\Users\billk\Desktop\Git\spirochete_vision\images


In [3]:
pd.read_csv('data_aug.csv', header=None)

,0,1,2,3,4,5,6,7,8,9,10
0,traingroup,filepath,object_class,xmin,ymin,TRX,TRY,xmax,ymax,BLX,BLY
1,TRAINING,1.jpg,1,0.7052083333333333,0.3885416666666667,NaN,NaN,0.7927083333333333,0.4177083333333333,NaN,NaN
2,TESTING,10.jpg,1,0.5208333333333334,0.4875,NaN,NaN,0.5760416666666667,0.559375,NaN,NaN
3,TRAINING,100.jpg,1,0.6096822995461422,0.6308623298033282,NaN,NaN,0.7246596066565809,0.7170953101361573,NaN,NaN
4,TRAINING,100.jpg,1,0.6671709531013615,0.7549167927382754,NaN,NaN,0.8290468986384266,0.8517397881996974,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
940,TRAINING,a7clock.jpg,1,0.045454545454545414,0.435064935064935,NaN,NaN,0.22077922077922074,0.551948051948052,NaN,NaN
941,TRAINING,a7clock.jpg,1,0.4285714285714286,0.3051948051948052,NaN,NaN,0.6038961038961039,0.435064935064935,NaN,NaN
942,TRAINING,a7clock.jpg,1,0.7532467532467533,0.09740259740259735,NaN,NaN,0.9805194805194806,0.24025974025974015,NaN,NaN
943,TRAINING,a8clock.jpg,1,0.3405405405405405,0.1891891891891892,NaN,NaN,0.5351351351351351,0.5459459459459459,NaN,NaN


Specify model

In [4]:
spec = model_spec.get('efficientdet_lite2')
spec.config.tflite_max_detections = 100

Use data loader

In [5]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('data_aug.csv')

Train model

In [6]:
model = object_detector.create(train_data, model_spec=spec, batch_size=9, epochs=50, train_whole_model=True, validation_data=validation_data)

Epoch 1/50
29/29 [==============================] - 70s 763ms/step - det_loss: 1.7604 - cls_loss: 1.0964 - box_loss: 0.0133 - reg_l2_loss: 0.0761 - loss: 1.8365 - learning_rate: 0.0096 - gradient_norm: 2.2546 - val_det_loss: 1.4035 - val_cls_loss: 0.8803 - val_box_loss: 0.0105 - val_reg_l2_loss: 0.0761 - val_loss: 1.4796
Epoch 2/50
29/29 [==============================] - 17s 573ms/step - det_loss: 1.1606 - cls_loss: 0.6443 - box_loss: 0.0103 - reg_l2_loss: 0.0761 - loss: 1.2368 - learning_rate: 0.0112 - gradient_norm: 3.9923 - val_det_loss: 1.4099 - val_cls_loss: 0.8307 - val_box_loss: 0.0116 - val_reg_l2_loss: 0.0762 - val_loss: 1.4861
Epoch 3/50
29/29 [==============================] - 17s 591ms/step - det_loss: 0.9149 - cls_loss: 0.5017 - box_loss: 0.0083 - reg_l2_loss: 0.0762 - loss: 0.9912 - learning_rate: 0.0112 - gradient_norm: 4.7006 - val_det_loss: 1.4161 - val_cls_loss: 0.7877 - val_box_loss: 0.0126 - val_reg_l2_loss: 0.0763 - val_loss: 1.4924
Epoch 4/50
29/29 [=============

Evaluate

In [7]:
model.evaluate(test_data)

1/1 [==============================] - 7s 7s/step



{'AP': 0.4258096,
 'AP50': 0.6186683,
 'AP75': 0.47948548,
 'APs': 0.4730033,
 'APm': 0.44951567,
 'APl': 0.28333333,
 'ARmax1': 0.39364865,
 'ARmax10': 0.6822973,
 'ARmax100': 0.6924324,
 'ARs': 0.76666665,
 'ARm': 0.69142854,
 'ARl': 0.9,
 'AP_/object_class': -1.0,
 'AP_/1': 0.625259,
 'AP_/2': 0.2263602}

In [8]:
model.export(export_dir='.')

In [9]:
model.evaluate_tflite('model.tflite', test_data)

36/36 [==============================] - 750s 21s/step



{'AP': 0.35893905,
 'AP50': 0.5247317,
 'AP75': 0.43326393,
 'APs': 0.45584157,
 'APm': 0.37762928,
 'APl': 0.28333333,
 'ARmax1': 0.32959458,
 'ARmax10': 0.5285135,
 'ARmax100': 0.53864866,
 'ARs': 0.7,
 'ARm': 0.5371429,
 'ARl': 0.9,
 'AP_/object_class': -1.0,
 'AP_/1': 0.5845867,
 'AP_/2': 0.13329145}

##### 